In [16]:
import json
import requests

from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace
from azure.storage.blob import BlobServiceClient 

# Connect to Azure ML Workspace
using our `azure-ml-adls` service principal

In [17]:
# import credentials saved in a local only file 
#    (can also use Azure Key Vault or environment variables for secure keys)
with open('./config.json') as f:
    creds = json.load(f)

svc_pr = ServicePrincipalAuthentication(
    tenant_id=creds['service_principal_tenant_id'],
    service_principal_id=creds['service_principal_app_id'],
    service_principal_password=creds['service_principal_password'])

ws = Workspace(
    subscription_id=creds['workspace_subscription_id'],
    resource_group=creds['workspace_resource_group'],
    workspace_name=creds['workspace_name'],
    auth=svc_pr # authorize connection using service principal
    )

# storage account connection string
storage_connect_str = creds['storage_connection_key']

print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace cait-projects at location westus2


# Get data
In order to calculate the 4 Big Growth Rates, we'll pull the necessary data from the company's 10K using [Financial Modeling Prep's API](https://financialmodelingprep.com/developer/docs) and upload it to our azure blob storage container we created in the [Azure Portal](https://portal.azure.com/) called "financial-data"

In [19]:
company_tickers = ['DPZ','ALK','MAR','SONY','GME','JNJ']

In [20]:
def upload_blob(data, blob_name, connect_str): 
    blob_service_client = BlobServiceClient.from_connection_string(connect_str) 
    blob_client = blob_service_client.get_blob_client(container="financial-data", blob=blob_name) 
    try: 
        # if file already exists, overwrite it with the new api response
        blob_client.get_blob_properties() 
        blob_client.delete_blob() 
        blob_client.upload_blob(data)
    except: 
        # otherwise just write the blob out to the container
        blob_client.upload_blob(data)

In [21]:
# for each company ticker in our list, get data and upload to our azure blob storage container 'financial-data'
for company in company_tickers:
    
    # Company Profile
    response = requests.get("https://financialmodelingprep.com/api/v3/profile/{}?apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        data = json.dumps(response.json())
        upload_blob(data, "company_profiles/company_profile_{}.json".format(company), storage_connect_str)
        
    # Income Sheet Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/income-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        data = json.dumps(response.json())
        upload_blob(data, "income_sheets/income_sheet_5years_{}.json".format(company), storage_connect_str)
        
    # Balance Sheet Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        data = json.dumps(response.json())
        upload_blob(data, "balance_sheets/balance_sheet_5years_{}.json".format(company), storage_connect_str)
        
    # Cash Flow Statement
    response = requests.get("https://financialmodelingprep.com/api/v3/cash-flow-statement/{}?limit=5&apikey={}".format(company, creds['fmp_api_key']))
    if response.status_code == 200:
        data = json.dumps(response.json())
        upload_blob(data, "cash_flow/cash_flow_5years_{}.json".format(company), storage_connect_str)        
        